In [9]:
#importing neccessary libraries

import pandas as pd
from geopy.geocoders import Nominatim
import folium
import time
import os
import numpy as np
import nltk
import seaborn as sns
import matplotlib.pyplot as plt


### Loading the datasets

In [11]:
df_sheet1 = pd.read_excel("../resources/dataset/final_clean_data.xlsx", sheet_name="Clean_Candidate_Donor_data")



In [12]:
df_sheet1.head()

,ID,Age,Horodateur,Niveau_scolaire,Genre_,Situation_Matrimoniale_(SM),Profession_,Arrondissement_de_résidence_,Quartier_de_Résidence_,Nationalité_,Religion_,A-t-il_(elle)_déjà_donné_le_sang_,Taux_d’hémoglobine_,ÉLIGIBILITÉ_AU_DON.
0,DONOR_1842,24,46:44.9,3,1,0,Etudiant (e),Douala 3,Yassa,Camerounaise,Chretien (Catholique),0,13.895459,1
1,DONOR_1843,23,44:19.0,3,1,0,Sans Emplois,Pas Précisé,Pas Précisé,Camerounaise,Chretien (Catholique),1,10.800000,0
2,DONOR_1844,41,46:54.3,0,1,2,Tailleur,Douala 1,NEW BELL,Camerounaise,Pas Précisé,0,13.895459,0
3,DONOR_1845,24,29:29.7,3,1,0,Etudiant,Douala 5,Ange raphael,Camerounaise,Musulman,0,15.000000,1
4,DONOR_1846,30,47:47.4,2,1,0,Militaire,Douala (Non précisé ),Beedi,Camerounaise,Adventiste,1,13.895459,1


In [13]:
df_sheet2 = pd.read_excel("../resources/dataset/final_clean_data.xlsx", sheet_name="Clean_Donor_Data")

In [14]:
df_sheet2.head()

,Sexe,Age,Type de donation,Groupe Sanguin ABO / Rhesus,Phenotype,Date
0,1,26,F,B+,"RH-2, RH+4, RH-3, RH+5, -kell1",2020-07-16
1,1,24,F,O+,"RH-2, RH+4, RH-3, RH+5, -kell1",2020-07-16
2,1,41,F,A+,"RH+2, RH+4, RH-3, RH+5, -kell1",2020-07-17
3,1,20,F,O+,"RH-2, RH+4, RH-3, RH+5, -kell1",2020-07-17
4,1,34,F,O+,"RH-2, RH+4, RH+3, RH+5, -kell1",2020-07-17


### Geospatial Map distribution of donors

In [16]:
geolocator = Nominatim(user_agent="geo_mapping")

def get_coordinates(location):
    try:
        time.sleep(1)  # Add delay to avoid rate limit issues
        loc = geolocator.geocode(location, timeout=10)  # Increase timeout
        if loc:
            return pd.Series([loc.latitude, loc.longitude])
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error for {location}: {e}")
        return pd.Series([None, None])

df_sheet1[['Latitude', 'Longitude']] = df_sheet1['Arrondissement_de_résidence_'].apply(get_coordinates)

Error for Douala (Non précisé ): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Douala+%28Non+pr%C3%A9cis%C3%A9+%29&format=json&limit=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002473FF9AE60>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 11001] getaddrinfo failed)"))
Error for Douala 5: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Douala+5&format=json&limit=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002473FF9B880>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 11001] getaddrinfo failed)"))
Error for Douala 3: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Douala+3&format=json&limit=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002473FF9B460>: Failed to resolve 'n

In [18]:
map_center = [df_sheet1['Latitude'].mean(), df_sheet1['Longitude'].mean()]
map_donors = folium.Map(location=map_center, zoom_start=12)

# Add points to map
for index, row in df_sheet1.dropna(subset=['Latitude', 'Longitude']).iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Arrondissement_de_résidence_'],
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(map_donors)

# Save or display
map_donors.save("donor_distribution.html")

In [19]:
import streamlit as st
# **Save Results**
df_sheet1.to_csv("donor_distribution_results.csv", index=False)
st.write(" Data saved successfully!")

2025-03-25 15:17:27.292 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 15:17:27.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 15:17:27.304 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 15:17:27.305 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
